## Libraries

In [92]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

from sklearn import set_config; set_config(display='diagram')
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer, make_column_selector,ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate

## Load Data 

In [93]:
df_22 = pd.read_csv('../raw_data/fifa22/players_22.csv')
df_21 = pd.read_csv('../raw_data/fifa22/players_21.csv')
df_20 = pd.read_csv('../raw_data/fifa22/players_20.csv')
df_19 = pd.read_csv('../raw_data/fifa22/players_19.csv')
df_18 = pd.read_csv('../raw_data/fifa22/players_18.csv')
df_17 = pd.read_csv('../raw_data/fifa22/players_17.csv')
df_16 = pd.read_csv('../raw_data/fifa22/players_16.csv')
df_15 = pd.read_csv('../raw_data/fifa22/players_15.csv')

/var/folders/tc/0tkkgmdd25n0rh3_pjfhw6mw0000gp/T/ipykernel_40742/3992483976.py:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_22 = pd.read_csv('../raw_data/fifa22/players_22.csv')
/var/folders/tc/0tkkgmdd25n0rh3_pjfhw6mw0000gp/T/ipykernel_40742/3992483976.py:5: DtypeWarning: Columns (81,82,83,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df_18 = pd.read_csv('../raw_data/fifa22/players_18.csv')
/var/folders/tc/0tkkgmdd25n0rh3_pjfhw6mw0000gp/T/ipykernel_40742/3992483976.py:7: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  df_16 = pd.read_csv('../raw_data/fifa22/players_16.csv')
/var/folders/tc/0tkkgmdd25n0rh3_pjfhw6mw0000gp/T/ipykernel_40742/3992483976.py:8: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  df_15 = pd.read_csv('../raw_data/fifa22/players_15.csv')


In [94]:
for index, col in enumerate(df_22.columns):
    print(f"{col:<26} | ", end="")
    if (index % 3 == 2):
        print("\n", end="")

sofifa_id                  | player_url                 | short_name                 | 
long_name                  | player_positions           | overall                    | 
potential                  | value_eur                  | wage_eur                   | 
age                        | dob                        | height_cm                  | 
weight_kg                  | club_team_id               | club_name                  | 
league_name                | league_level               | club_position              | 
club_jersey_number         | club_loaned_from           | club_joined                | 
club_contract_valid_until  | nationality_id             | nationality_name           | 
nation_team_id             | nation_position            | nation_jersey_number       | 
preferred_foot             | weak_foot                  | skill_moves                | 
international_reputation   | work_rate                  | body_type                  | 
real_face                  | rel

In [95]:
to_drop = ['club_logo_url','nation_flag_url','club_flag_url','nation_logo_url','player_face_url','dob','player_url',
           'real_face','nation_jersey_number','nation_position','club_loaned_from','long_name','player_url'
          ,'ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm',
           'rm','lwb','ldm', 'cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk','club_jersey_number','nationality_id','club_jersey_number']



In [96]:
df22 = df_22.drop(to_drop, axis = 1)
df21 = df_21.drop(to_drop, axis = 1)
df20 = df_20.drop(to_drop, axis = 1)
df19 = df_19.drop(to_drop, axis = 1)
df18 = df_18.drop(to_drop, axis = 1)
df17 = df_17.drop(to_drop, axis = 1)
df16 = df_16.drop(to_drop, axis = 1)
df15 = df_15.drop(to_drop, axis = 1)

In [97]:
def is_bench(d):
    if d in ("SUB", "RES", ""):
        return True 
    return False

In [98]:
dfs = [df18, df19, df20, df21,df22,df17,df16,df15]

for frame in dfs:
    frame['is_bench'] = frame['club_position'].apply(is_bench)
    frame['potential_diff'] = frame['potential'] - frame['overall']
    frame[['att_rate', 'def_rate']] = frame['work_rate'].str.split('/', 1, expand=True)

In [99]:
pos22 = df22['player_positions']
pos21 = df21['player_positions']
pos20 = df20['player_positions']
pos19 = df19['player_positions']
pos18 = df18['player_positions']
pos17 = df17['player_positions']
pos16 = df16['player_positions']
pos15 = df15['player_positions']


pref_pos_22 = []
pref_pos_21 = []
pref_pos_20 = []
pref_pos_19 = []
pref_pos_18 = []
pref_pos_17 = []
pref_pos_16 = []
pref_pos_15 = []

In [100]:
pos = [pos18, pos19, pos20, pos21,pos22,pos17,pos16,pos15]

for i in pos22:
    a = i.split(',')
    pref_pos_22.append(a[0])
df22['prefered_pos'] = pref_pos_22

for i in pos21:
    a = i.split(',')
    pref_pos_21.append(a[0])
df21['prefered_pos'] = pref_pos_21


for i in pos20:
    a = i.split(',')
    pref_pos_20.append(a[0])
df20['prefered_pos'] = pref_pos_20

for i in pos19:
    a = i.split(',')
    pref_pos_19.append(a[0])
df19['prefered_pos'] = pref_pos_19

for i in pos18:
    a = i.split(',')
    pref_pos_18.append(a[0])
df18['prefered_pos'] = pref_pos_18

for i in pos17:
    a = i.split(',')
    pref_pos_17.append(a[0])
df17['prefered_pos'] = pref_pos_17

for i in pos16:
    a = i.split(',')
    pref_pos_16.append(a[0])
df16['prefered_pos'] = pref_pos_16

for i in pos15:
    a = i.split(',')
    pref_pos_15.append(a[0])
df15['prefered_pos'] = pref_pos_15  



In [101]:
df22 = df22.drop('player_positions', axis = 1)
df21 = df21.drop('player_positions', axis = 1)
df20 = df20.drop('player_positions', axis = 1)
df19 = df19.drop('player_positions', axis = 1)
df18 = df18.drop('player_positions', axis = 1)
df17 = df17.drop('player_positions', axis = 1)
df16 = df16.drop('player_positions', axis = 1)
df15 = df15.drop('player_positions', axis = 1)

df22['prefered_pos'] = df22['prefered_pos'].astype('category')
df21['prefered_pos'] = df21['prefered_pos'].astype('category')
df20['prefered_pos'] = df20['prefered_pos'].astype('category')
df19['prefered_pos'] = df19['prefered_pos'].astype('category')
df18['prefered_pos'] = df18['prefered_pos'].astype('category')
df17['prefered_pos'] = df17['prefered_pos'].astype('category')
df16['prefered_pos'] = df16['prefered_pos'].astype('category')
df15['prefered_pos'] = df15['prefered_pos'].astype('category')




## Join Dataframes

In [102]:
df_all = df22.join(df21, rsuffix='_21')
df_all = df_all.join(df20, rsuffix='_20')
df_all = df_all.join(df19, rsuffix='_19')
df_all = df_all.join(df18, rsuffix='_18')
df_all = df_all.join(df17, rsuffix='_17')
df_all = df_all.join(df16, rsuffix='_16')
df_all = df_all.join(df15, rsuffix='_15')

In [103]:
df_all['is_bench_21'] = df_all['is_bench_21'].astype('bool')
df_all['is_bench_20'] = df_all['is_bench_20'].astype('bool')
df_all['is_bench_19'] = df_all['is_bench_19'].astype('bool')
df_all['is_bench_18'] = df_all['is_bench_18'].astype('bool')
df_all['is_bench_17'] = df_all['is_bench_17'].astype('bool')
df_all['is_bench_16'] = df_all['is_bench_16'].astype('bool')
df_all['is_bench_15'] = df_all['is_bench_15'].astype('bool')

In [104]:
df_all.select_dtypes("bool").nunique()

is_bench       2
is_bench_21    2
is_bench_20    2
is_bench_19    2
is_bench_18    2
is_bench_17    2
is_bench_16    2
is_bench_15    2
dtype: int64

In [105]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 584 entries, sofifa_id to prefered_pos_15
dtypes: bool(8), category(8), float64(420), int64(44), object(104)
memory usage: 83.7+ MB


19239

In [106]:
# df_all.select_dtypes("float64").nunique()

df_all.work_rate.value_counts()

Medium/Medium    10015
High/Medium       3661
Medium/High       1880
High/High         1099
Medium/Low         810
High/Low           808
Low/Medium         466
Low/High           455
Low/Low             45
Name: work_rate, dtype: int64

In [110]:
feat_categorical_nunique = df_all.select_dtypes("object").nunique()
feat_categorical_nunique

short_name          18145
club_name             701
league_name            55
club_position          29
club_joined          1845
                    ...  
body_type_15           10
player_tags_15         59
player_traits_15      829
att_rate_15             3
def_rate_15             3
Length: 96, dtype: int64

In [111]:
feat_categorical_small = feat_categorical_nunique[feat_categorical_nunique < 10]
feat_categorical_small

preferred_foot       2
att_rate             3
def_rate             3
preferred_foot_21    2
att_rate_21          3
def_rate_21          3
preferred_foot_20    2
att_rate_20          3
def_rate_20          3
preferred_foot_19    2
att_rate_19          3
def_rate_19          3
preferred_foot_18    2
att_rate_18          3
def_rate_18          3
preferred_foot_17    2
att_rate_17          3
def_rate_17          3
preferred_foot_16    2
att_rate_16          3
def_rate_16          3
preferred_foot_15    2
att_rate_15          3
def_rate_15          3
dtype: int64

In [109]:
wk_drop = ['work_rate','work_rate_21','work_rate_20','work_rate_19',
           'work_rate_18','work_rate_17','work_rate_16','work_rate_15']

df_all = df_all.drop(wk_drop, axis = 1)

## Pipeline

In [ ]:
# Preprocessor

num_transformer = make_pipeline(SimpleImputer(), StandardScaler())


cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)
cat_col = make_column_selector(dtype_include=['object','bool'])

preproc = make_column_transformer(
    (num_transformer, make_column_selector(dtype_include=['float64'])),
    (cat_transformer, make_column_selector(dtype_include=['object','bool'])),
    remainder=‘passthrough’)



# Add Estimator
pipe = make_pipeline(preproc, Ridge())
pipe